# Zestaw zadań: Równania różniczkowe - spectral bias
## Zadanie 1
### Autor: Artur Gęsiarz

In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

### Definicja analitycznego rozwiązania

In [ ]:
def exact_solution(x, w):
    return (1/w) * torch.sin(w * x)

### Definicja sieci neuronowej

In [ ]:
class FCN(nn.Module):
    def __init__(self, N_INPUT, N_OUTPUT, N_HIDDEN, N_LAYERS):
        super().__init__()
        activation = nn.Tanh
        self.fcs = nn.Sequential(*[
                        nn.Linear(N_INPUT, N_HIDDEN),
                        activation()])
        self.fch = nn.Sequential(*[
                        nn.Sequential(*[
                            nn.Linear(N_HIDDEN, N_HIDDEN),
                            activation()]) for _ in range(N_LAYERS-1)])
        self.fce = nn.Linear(N_HIDDEN, N_OUTPUT)
    def forward(self, x):
        x = self.fcs(x)
        x = self.fch(x)
        x = self.fce(x)
        return x

### Funkcja treningowa PINN

In [ ]:
def train_PINN(model, x_boundary, x_physics, w, epochs=50000, lr=0.001):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    losses = []

    for i in range(epochs):
        optimizer.zero_grad()

        # Koszt warunku początkowego
        u0 = model(x_boundary)
        loss_ic = u0**2

        # Koszt rezydualny
        u = model(x_physics)
        du_dx = torch.autograd.grad(u, x_physics, torch.ones_like(u), create_graph=True)[0]
        residual = du_dx - torch.cos(w * x_physics)
        loss_r = torch.mean(residual**2)

        # Łączny koszt
        loss = loss_ic + loss_r
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        if i % 5000 == 0:
            print(f'Epoch {i}, Loss: {loss.item()}')

    return losses